# Titanic disaster model

https://www.kaggle.com/competitions/titanic

https://www.kaggle.com/code/alexisbcook/titanic-tutorial/notebook

https://www.overleaf.com/read/fqxnygwqtnjs#fbe3c8

### Seting up environment

In [5]:
import pandas as pd
# found raw files on github // KK;23112023 -> no need? -> #download training set and upload it into files (cataloge icon)
gh_path = 'https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/'
df_train = pd.read_csv(gh_path + 'train.csv')
df_test = pd.read_csv(gh_path + 'test.csv')
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Feature engeneering
#### To do:
- change values of sex for binary (f.e. male = 0, female = 1),
- reduce number of values (drop columns: Name, Ticket, ?Fare?, Cabin, Embarked),
- drop keys with NaN values

A: Fare value can carry some information about Cabin, if we want to include passenger placement in model.

### Perceptron

#### To do:
- define weight vector W (imo randomly) and training set X
- definition of perceptron class (linear, binary[if survived would be changed to -1 and 1] or/and logistic). That would include: activation function, loss function, fiting (gradient descent/newton-rap), all can be copied and slightly modified form collab "logistic_regression_and_svm",
- spliting training dataset and look for bad predictions,
- comparason with scikit-learn method

### Suport Vector Machine
#### To do: 
- define 6D weight vector (Pclass,Sex,Age,SibSp,Parch,Fare), loss function (Lagrange function), calculate width of the margin (constraint equation), Optimize and get the values of $\vec{\alpha}$, w and bias b.  kernel transformation(s),
- soft- (leanring rate (regulatization) C) and hard-margin,
- spliting training dataset and look for bad predictions,
- compare with scikit-learn method.

$$ L(\vec{\alpha}) = \sum _i \alpha _i - \frac{1}{2} \sum _{i,j} \alpha _i \alpha _j y_i y_j \vec{X}_i\cdot \vec{X} _j. $$

##### [`REMARK`] It's crucial to keep track of this equation and the dot product within it, as it becomes highly significant later on, especially when dealing with nonlinear data.

#### Random Forest Model
##### To do:
- arono

##### Random forest (kaggle example)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
test_data = pd.read_csv('titanic_data/test.csv')
y = df_train["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(df_train[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

#### Analysis of results